In [5]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import os
from datetime import datetime
import qrcode

def add_title(img, title, font_scale=0.7, thickness=2):
    cv2.putText(img, title, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

def attempt_decode(frame):
    detected_codes = []
    decoded_objects = decode(frame)
    if decoded_objects:
        for obj in decoded_objects:
            data = obj.data.decode("utf-8")
            code_type = obj.type
            print(f"Decoded Data: {data}, Type: {code_type}")
            points = obj.polygon
            if len(points) >= 4:
                points = np.array(points, dtype=np.int32)
                cv2.polylines(frame, [points], True, (255, 0, 0), 3)
                cv2.putText(frame, data, (obj.rect.left, obj.rect.top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            detected_codes.append((data, code_type))
    return frame, detected_codes

def save_codes_to_file(detected_codes):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    file_name = f"detected_codes_{timestamp}.txt"
    with open(file_name, "w") as file:
        for data, code_type in detected_codes:
            file.write(f"Type: {code_type}, Data: {data}\n")

def generate_qr_code(data, filename):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img.save(filename)

def main():
    cam = cv2.VideoCapture(0)
    last_save_time = datetime.now()

    # Generate QR code from data
    qr_data = "https://www.example.com"
    qr_filename = "generated_qr_code.png"
    generate_qr_code(qr_data, qr_filename)

    while True:
        success, frame = cam.read()
        if not success:
            print("Failed to grab frame")
            break

        # Attempt decoding QR code
        processed_frame, detected_codes = attempt_decode(frame.copy())

        # Save detected codes to a file if not already saved within last 5 seconds
        if detected_codes and (datetime.now() - last_save_time).total_seconds() > 5:
            save_codes_to_file(detected_codes)
            last_save_time = datetime.now()

        # Add titles to images
        add_title(frame, 'Original Detector')

        # Resize images for display
        frame_resized = cv2.resize(frame, (640, 480))
        processed_frame_resized = cv2.resize(processed_frame, (640, 480))

        # Concatenate images for display
        grid = np.hstack((frame_resized, processed_frame_resized))

        cv2.imshow('Qrcode & Barcode Detection', grid)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
